In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold



In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('F:/notebook_working/kaggle_compe/train.csv').fillna(' ')
test = pd.read_csv('F:/notebook_working/kaggle_compe/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])



In [3]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    min_df=3,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use char"
print(msg.format(len(word_vectorizer.get_feature_names())))


There are 78133 tokens in Comment_text if we use char


In [4]:
char_vectorizer = TfidfVectorizer(
    #norm=None,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1,2), 
    #non_negative=True,
    #min_df = 0.00009)
    max_features=10000)
    
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

msg = "There are {} tokens in Comment_text if we use char"
print(msg.format(len(char_vectorizer.get_feature_names())))


MemoryError: 

In [14]:
#char_vectorizer.get_feature_names()

In [9]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.971621213096
CV score for class severe_toxic is 0.986992126684
CV score for class obscene is 0.985432960874
CV score for class threat is 0.981061078101
CV score for class insult is 0.977306420841
CV score for class identity_hate is 0.971915652777
Total CV score is 0.979054908729


In [ ]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

In [7]:
print train_word_features.shape
print train_char_features.shape
print test_word_features.shape
print test_char_features.shape

(159571, 318196)
(159571, 4116)
(153164, 318196)
(153164, 4116)


In [ ]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    #X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    skf=StratifiedKFold(n_splits=3, shuffle=False)
    #skf=StratifiedKFold(n_splits=3, shuffle=True)
    for train_indices, valid_indices in skf.split(train_word_features, train_target):
        X_train, y_train = train_word_features[train_indices], train_target[train_indices]
        X_valid, y_valid = train_word_features[valid_indices], train_target[valid_indices]
    
    classifier = LogisticRegression()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

In [13]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)

    classifier = SVC()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.953617374677
CV score for class severe_toxic is 0.878066506914
CV score for class obscene is 0.978812954813
CV score for class threat is 0.885887012223
CV score for class insult is 0.95942138282
CV score for class identity_hate is 0.871775257044
Total CV score is 0.921263414748


In [9]:
submission[class_name] = classifier.predict_proba(test_features)[:, 1]
submission.to_csv('submission.csv', index=False)